In [1]:
%pip install h2o

Note: you may need to restart the kernel to use updated packages.


In [2]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

In [3]:
# Start the H2O cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu220.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu220.04.1, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp4v32pstf
  JVM stdout: /tmp/tmp4v32pstf/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp4v32pstf/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.2
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_unknownUser_2fp10e
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [4]:
# Import train/test set into H2O
train = h2o.import_file("/kaggle/input/playground-series-s4e6/train.csv")
test = h2o.import_file("/kaggle/input/playground-series-s4e6/test.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.head()

id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,1,1,9238,1,1,126,1,1,19,5,5,122.6,0,0,0,1,0,1,18,0,0,6,6,6,14.5,0,0,6,7,6,12.4286,0,11.1,0.6,2.02,Graduate
1,1,17,1,9238,1,1,125,1,19,19,9,9,119.8,1,0,0,1,0,0,18,0,0,6,8,4,11.6,0,0,6,9,0,0,0,11.1,0.6,2.02,Dropout
2,1,17,2,9254,1,1,137,1,3,19,2,3,144.7,0,0,0,1,1,0,18,0,0,6,0,0,0,0,0,6,0,0,0,0,16.2,0.3,-0.92,Dropout
3,1,1,3,9500,1,1,131,1,19,3,3,2,126.1,1,0,0,1,0,1,18,0,0,7,9,7,12.5913,0,0,8,11,7,12.82,0,11.1,0.6,2.02,Enrolled
4,1,1,2,9500,1,1,132,1,19,37,4,9,120.1,1,0,0,1,0,0,18,0,0,7,12,6,12.9333,0,0,7,12,6,12.9333,0,7.6,2.6,0.32,Graduate
5,1,39,1,171,1,19,133.1,1,19,19,1,1,100,0,0,0,1,1,0,24,0,0,0,0,0,0,0,0,0,0,0,0,0,13.9,-0.3,0.79,Dropout
6,1,44,1,9085,1,39,130,1,37,37,9,6,130,1,0,0,1,0,0,21,0,0,5,7,4,12.75,0,0,5,8,5,13.25,0,12.4,0.5,1.79,Graduate
7,1,1,1,9773,1,1,130,1,19,37,4,5,133.9,1,0,0,1,1,0,18,0,0,6,6,5,13,0,0,6,6,6,13.6667,0,10.8,1.4,1.74,Graduate
8,1,39,1,9003,1,1,133.1,1,19,19,9,9,130,1,0,1,0,1,0,24,0,0,6,8,0,0,0,0,6,10,0,0,0,16.2,0.3,-0.92,Dropout
9,1,1,6,9500,1,1,135,1,37,37,4,9,128,1,0,0,1,0,1,18,0,0,7,7,6,12.9667,0,0,7,7,6,12.9667,0,7.6,2.6,0.32,Graduate


In [6]:
test.head()

id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
76518,1,1,1,9500,1,1,141,1,3,1,2,3,152.1,0,0,0,0,0,0,18,0,0,7,0,0,0,0,0,8,0,0,0,0,13.9,-0.3,0.79
76519,1,1,1,9238,1,1,128,1,1,19,4,5,116.5,0,0,0,1,0,0,19,0,0,6,7,6,14.8571,0,0,6,6,6,13.5,0,11.1,0.6,2.02
76520,1,1,1,9238,1,1,118,1,1,19,4,9,114.2,0,0,0,1,0,1,18,0,0,6,11,6,12,0,0,6,11,5,11,0,15.5,2.8,-4.06
76521,1,44,1,9147,1,39,130,1,1,19,3,3,130,0,0,0,1,0,1,23,0,2,6,15,5,11.5,0,3,8,14,5,11,0,8.9,1.4,3.51
76522,1,39,1,9670,1,1,110,1,1,37,5,5,106,1,0,0,1,0,0,26,0,0,6,9,3,11,0,0,6,9,4,10.6667,2,7.6,2.6,0.32
76523,1,17,4,9238,1,1,137,1,1,3,3,3,127.4,1,0,0,1,0,0,18,0,0,6,7,6,15,0,0,6,7,6,14.4286,0,11.1,0.6,2.02
76524,1,1,1,9773,1,1,126,1,1,37,4,3,118.7,0,0,0,1,0,0,18,0,0,6,6,6,12.6667,0,0,6,6,6,13.3333,0,12.7,3.7,-1.7
76525,1,44,1,9003,1,39,150,1,19,38,9,8,150,1,0,0,1,0,0,21,0,0,6,8,6,12.75,0,0,6,6,6,12.6667,0,13.9,-0.3,0.79
76526,1,1,2,9773,1,1,131,1,38,38,9,9,132.3,1,0,1,0,1,1,18,0,0,6,8,4,10.75,0,0,6,8,5,11,0,8.9,1.4,3.51
76527,1,1,1,9254,1,1,111,1,19,38,3,3,109.3,1,0,0,1,1,0,19,0,0,6,8,6,12.8333,0,0,6,6,6,13.1667,0,11.1,0.6,2.02


In [7]:
# Specify target and predictors
y = "Target"
x = train.columns
x.remove(y)

In [8]:
# Convert target column to factor
train[y] = train[y].asfactor()

In [9]:
# Run AutoML for 10 base models
aml = H2OAutoML(max_models=10, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),10/10
# GBM base models (used / total),4/4
# XGBoost base models (used / total),3/3
# DRF base models (used / total),2/2
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [10]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_AllModels_1_AutoML_1_20240608_182729,0.208698,0.436713,0.372078,0.138442
StackedEnsemble_BestOfFamily_1_AutoML_1_20240608_182729,0.208759,0.437916,0.372535,0.138782
XGBoost_3_AutoML_1_20240608_182729,0.211601,0.436433,0.370107,0.136979
GBM_3_AutoML_1_20240608_182729,0.211729,0.438943,0.370786,0.137483
GBM_2_AutoML_1_20240608_182729,0.21227,0.437896,0.370372,0.137175
GBM_1_AutoML_1_20240608_182729,0.2126,0.438883,0.370621,0.13736
GBM_4_AutoML_1_20240608_182729,0.215176,0.442782,0.372155,0.138499
XGBoost_2_AutoML_1_20240608_182729,0.219039,0.451044,0.374055,0.139917
DRF_1_AutoML_1_20240608_182729,0.220498,0.480731,0.382135,0.146027
XGBoost_1_AutoML_1_20240608_182729,0.221771,0.459349,0.376163,0.141499


In [22]:
#Predict target from test data
pred_h2o = aml.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [24]:
#Convert h2o data to dataframe
test_df = test.as_data_frame()
pred_df = pred_h2o.as_data_frame()

#Save prediction to csv file
pred = pd.DataFrame()
pred['id'] = test_df['id'] 
pred['Target'] = pred_df['predict'] 
pred.to_csv('/kaggle/working/submission.csv',index=False)
pred

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O f

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Enrolled
4,76522,Enrolled
...,...,...
51007,127525,Dropout
51008,127526,Dropout
51009,127527,Dropout
51010,127528,Dropout
